In [1]:
import tensorflow as tf
import numpy as np
import os

def read_data():
    reader = tf.TFRecordReader()
    train_name = ["train-0.tfrecords", "train-1.tfrecords", "train-2.tfrecords", "train-3.tfrecords", "train-4.tfrecords", "train-5.tfrecords", "train-6.tfrecords", "train-7.tfrecords"]
#     train_name = ["validation.tfrecords"]
    xx = []
    for i in train_name:
        file = os.path.join("TFRecord/",i)
        xx.append(file)
    filename_queue = tf.train.string_input_producer(xx)

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
            serialized_example,
            features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    labels = tf.cast(features['label'], tf.int32)
    images = tf.decode_raw(features['image_raw'], tf.uint8)
    images = tf.reshape(images,[32,48])
    


    with tf.Session() as sess:
        images_batch, labels_batch = tf.train.batch([images, labels],
                                                            batch_size=256,
                                                            capacity=500
                                                            )
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        image, label = sess.run([images_batch, labels_batch])
        coord.request_stop()
        coord.join(threads)
    return images_batch, labels_batch
        
        
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def avg_pool_2x2(x):
    """avg_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def deepnn(x):
    # input
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 32, 48, 1])
    
    # First convolutional layer
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([3, 7, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        
        W_conv2 = weight_variable([3, 7, 32, 48])
        b_conv2 = bias_variable([48])
        h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
        

    # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv2)
        h_pool1 = tf.nn.dropout(h_pool1, keep_prob)
        

    # second convolutional layer
    with tf.name_scope("conv2"):
        W_conv3 = weight_variable([3,3,48,64])
        b_conv3 = bias_variable([64])
        h_conv3 = tf.nn.relu(conv2d(h_pool1, W_conv3) + b_conv3)
        
        W_conv4 = weight_variable([3,3,64,72])
        b_conv4 = bias_variable([72])
        h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)
        
    # max_pool
    with tf.name_scope("pool2"):
        h_pool2 = avg_pool_2x2(h_conv4)
        h_pool2 = tf.nn.dropout(h_pool2, keep_prob)
        
        
    # third convolutional layer
    with tf.name_scope("conv3"):
        W_conv5 = weight_variable([3,3,72,128])
        b_conv5 = bias_variable([128])
        h_conv5 = tf.nn.relu(conv2d(h_pool2, W_conv5) + b_conv5)
        
    # pool
    with tf.name_scope("pool3"):
        h_pool3 = avg_pool_2x2(h_conv5)
        h_pool3 = tf.nn.dropout(h_pool3, keep_prob)

  # Fully connected layer 1 
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([4*6*128, 256])
        b_fc1 = bias_variable([256])
        
        h_pool3_flat = tf.reshape(h_pool3, [-1, 4*6*128])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
    
  # Dropout - controls the complexity of the model, prevents co-adaptation of features.
    with tf.name_scope('dropout'):
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
        
    # Fully connected layer 2    
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([256, 4*11])
        b_fc2 = bias_variable([4*11])

        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    return y_conv, keep_prob


# Import data
images_batch, labels_batch = read_data()

# Create the model
x = tf.placeholder(tf.float32, [None, 32, 48])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 4, 11])

# keep_prob
keep_prob = tf.placeholder(tf.float32)

def onhot(lab):
    arr = np.zeros([256,4,11])
    for a, b in enumerate(lab):
        y = list(str(b))
        for i in range(len(y)):
            y[i] = int(y[i])
        
        for i in range(4):
            if i < len(y):
                arr[a,i,y[i]] = 1
            else:
                arr[a,i,10] = 1
    return arr

# Build the graph for the deep net
y_conv, keep_prob = deepnn(x)

yy = tf.reshape(y_conv,[-1,4,11])


with tf.name_scope('loss'):
    cross_entropy1 =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_[:,0],
                                                        logits=yy[:,0]))
    cross_entropy2 =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_[:,1],
                                                        logits=yy[:,1]))
    cross_entropy3 =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_[:,2],
                                                        logits=yy[:,2]))
    cross_entropy4 =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_[:,3],
                                                        logits=yy[:,3]))
    cross_entropy = (cross_entropy1 + cross_entropy2 + cross_entropy3 + cross_entropy4) / 4.
    aa = tf.summary.scalar("loss",cross_entropy)
    

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

with tf.name_scope('accuracy'):
    #correct_prediction = [tf.equal(tf.argmax(tf.nn.softmax(yy[None, i]), 1), tf.argmax(y_[None, i], 1)) for i in range(4)]
    correct_prediction = tf.equal(tf.argmax(tf.reshape(y_conv,[-1,4,11]), 2), tf.argmax(tf.reshape(y_, [-1,4,11]), 2))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    
    accuracy = tf.reduce_mean(correct_prediction)

ckpt_dir = "./test_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
# saver
saver = tf.train.Saver()


In [3]:

with tf.Session() as sess:
    # 初始化变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    for epoch in range(20):
        img, lab = sess.run([images_batch, labels_batch])
        labe = onhot(lab)
            
        my_loss, _, accuracy_validation, pre = sess.run([cross_entropy, train_step, accuracy, yy], feed_dict={x: img / 255, y_: labe, keep_prob: 1})
        
#         for i in range(30):
#             print(np.argmax(pre[i],1))
#             print(np.argmax(labe[i],1))
#             print("_____________________________________")
        
        if epoch % 1 == 0:
            print('epoch %d, loss is %g validation accuracy %g' % (
                epoch,
                float(my_loss),
                float(accuracy_validation)))
    coord.request_stop()
    coord.join(threads)


./test_dir/model.ckpt
INFO:tensorflow:Restoring parameters from ./test_dir/model.ckpt
epoch 0, loss is 0.0260106 validation accuracy 0.988281
epoch 1, loss is 0.0180938 validation accuracy 0.996094
epoch 2, loss is 0.0110382 validation accuracy 0.998047
epoch 3, loss is 0.0158613 validation accuracy 0.994141
epoch 4, loss is 0.0156227 validation accuracy 0.998047
epoch 5, loss is 0.0182083 validation accuracy 0.994141
epoch 6, loss is 0.0214361 validation accuracy 0.994141
epoch 7, loss is 0.016967 validation accuracy 0.994141
epoch 8, loss is 0.017105 validation accuracy 0.995117
epoch 9, loss is 0.0179023 validation accuracy 0.99707
epoch 10, loss is 0.0145826 validation accuracy 0.994141
epoch 11, loss is 0.0128047 validation accuracy 0.998047
epoch 12, loss is 0.0124751 validation accuracy 0.99707
epoch 13, loss is 0.0163951 validation accuracy 0.99707
epoch 14, loss is 0.00856568 validation accuracy 0.999023
epoch 15, loss is 0.0138171 validation accuracy 0.995117
epoch 16, loss i

In [2]:
with tf.Session() as sess:
    # 初始化变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    # tensorboard
    writer = tf.summary.FileWriter('graph', sess.graph)
    merged = tf.summary.merge([aa])
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    for epoch in range(1500):
        img, lab = sess.run([images_batch, labels_batch])
        labe = onhot(lab)
        
#         %matplotlib inline
#         import matplotlib.pyplot as plt
#         plt.imshow(img[0].reshape([32, 48]))
#         print(lab[0])
#         print(labe[0])
#         break

        my_loss, _, accuracy_validation, summary = sess.run([cross_entropy, train_step, accuracy, merged], feed_dict={x: img / 255., y_: labe, keep_prob: 0.75})
        
        writer.add_summary(summary,epoch)
        
        if epoch % 50 == 0:
            saver.save(sess,ckpt_dir + "/model.ckpt")
            print('epoch %d, loss is %g validation accuracy %g' % (
                epoch,
                float(my_loss),
                float(accuracy_validation)))
            
    coord.request_stop()
    coord.join(threads)

./test_dir/model.ckpt
INFO:tensorflow:Restoring parameters from ./test_dir/model.ckpt
epoch 0, loss is 0.143894 validation accuracy 0.946289
epoch 50, loss is 0.121993 validation accuracy 0.956055
epoch 100, loss is 0.124289 validation accuracy 0.956055
epoch 150, loss is 0.0916146 validation accuracy 0.967773
epoch 200, loss is 0.144487 validation accuracy 0.948242
epoch 250, loss is 0.169898 validation accuracy 0.942383
epoch 300, loss is 0.103348 validation accuracy 0.960938
epoch 350, loss is 0.113049 validation accuracy 0.957031
epoch 400, loss is 0.158303 validation accuracy 0.947266
epoch 450, loss is 0.11112 validation accuracy 0.963867
epoch 500, loss is 0.112569 validation accuracy 0.958984
epoch 550, loss is 0.0937502 validation accuracy 0.970703
epoch 600, loss is 0.0923246 validation accuracy 0.96582
epoch 650, loss is 0.118696 validation accuracy 0.954102
epoch 700, loss is 0.112628 validation accuracy 0.957031
epoch 750, loss is 0.0751323 validation accuracy 0.964844
epo

In [2]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

a = os.listdir("./image")
data = []
for i,j in enumerate(a):
    files_name = os.path.join("./image",j)
    a_image = Image.open(files_name).convert('L')
    changed_image = a_image.resize((48,32),Image.ANTIALIAS)
    image_raw = np.array(changed_image)
#     plt.imshow(image_raw,cmap="gray")
#     plt.show()
    data.append(image_raw/255.)

with tf.Session() as sess:
    # 初始化变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    output = sess.run([yy],feed_dict = {x:data, keep_prob:1})
    print(np.argmax(output,3))
    with open("data/yanzhengbin.txt","w") as f:
        for i in range(len(output)):
            f.write(str(np.argmax(output, 3)[i])+'\n')
    coord.request_stop()
    coord.join(threads)

./test_dir/model.ckpt
INFO:tensorflow:Restoring parameters from ./test_dir/model.ckpt
[[[ 6  6  7 10]
  [ 6  8  3 10]
  [ 8  3  8  4]]]


In [3]:
with open("data/yanzhengbin.txt","rt") as f:
        x = f.readlines()
        print(x)

['[[ 6  6  7 10]\n', ' [ 6  8  3 10]\n', ' [ 8  3  8  4]]\n']
